# Architecture

![Screenshot](./flow_diagram.png)

In [12]:
!pip install langchain_huggingface

  Obtaining dependency information for langchain_huggingface from https://files.pythonhosted.org/packages/9d/f8/77a303ddc492f6eed8bf0979f2bc6db4fa6eb1089c5e9f0f977dd87bc9c2/langchain_huggingface-0.1.2-py3-none-any.whl.metadata
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Obtaining dependency information for huggingface-hub>=0.23.0 from https://files.pythonhosted.org/packages/93/27/1fb384a841e9661faad1c31cbfa62864f59632e876df5d795234da51c395/huggingface_hub-0.30.2-py3-none-any.whl.metadata
  Obtaining dependency information for sentence-transformers>=2.6.0 from https://files.pythonhosted.org/packages/45/2d/1151b371f28caae565ad384fdc38198f1165571870217aedda230b9d7497/sentence_transformers-4.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers>=0.19.1 from https://files.pythonhosted.org/packages/e6/b6/072a8e053ae600dcc2ac0da81a23548e3b523301a442a6ca900e92ac35be/tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata
  Obtaining depe


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader #To load the csv file (data containing companys faq)
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint,HuggingFaceEmbeddings

## Step-1 Load the data (company_Q1.csv)

#### load the CSV file that Contains the FAQ question regarding the company

In [ ]:
file_path = "./company_QA.csv" #Path to the file
loader = CSVLoader(file_path=file_path) #CSVLoader to load the CSV file
docs = []
for doc in loader.lazy_load(): #Perfom lazy load
    docs.append(doc)


In [ ]:
len(docs) # Total number of document object

50

In [10]:
docs[0] #first document object

Document(metadata={'source': './company_QA.csv', 'row': 0}, page_content="Question: Where is the company's headquarters located?\nAnswer: Our headquarters is located in San Francisco, California. Nestled in the vibrant downtown area, it provides easy access to public transport and major city landmarks.")

### Note : No need to performing chunking since each document object is a single row of the CSV file

### Step-2 Load the LLM and Embedding model from HuggingFace

In [ ]:
# Load the LLM model (Mistral-7b)
